In [ ]:
!pip install oauthlib
!pip install requests_oauthlib
!pip install sentinelhub

In [5]:
import pandas as pd
from PIL import Image
from datetime import datetime, timedelta
import requests
from data_mining.get_sat2_images import call_sentinel
# from data_mining.read_firenews import fetch_fire_news
import data_mining.read_weather as weather

# read fire history

## fetch from fire news

In [ ]:
# start_date="2022-05-01"
# end_date="2025-06-10"
# fire_data=fetch_fire_news(start_date,end_date,verbose=True)
# fetch_fire_news(start_date,end_date,save=True)

## or read from file

In [6]:
# fire_data= pd.read_csv("fire_news.csv")
fire_data = pd.read_csv("fire_labels/fire_labels_final.csv")
fire_data = fire_data[["latitude", "longitude", "date"]][::-1]
# fire_data.rename(columns={'latitude_clean':'latitude','longitude_clean':'longitude'},inplace=True)
print(len(fire_data))
fire_data.drop_duplicates(inplace=True)
print(len(fire_data))

1147
610


# get images from sentinel 2

In [30]:
# Your client credentials
client_id = '4d093505-d867-4ae4-90b0-468eb78dd9af'
client_secret = 'UHSroMy3xUHqh0SakSXrPvCejglb2eyH'

In [31]:
for index, row in fire_data.iterrows():
    day = row["date"]
    day_date = datetime.strptime(day, "%Y-%m-%d")
    yesterday = (day_date - timedelta(days=1)).strftime("%Y-%m-%d")
    time_interval=(yesterday, day)
    sucsess = call_sentinel(client_id,client_secret,row,time_interval,save=True)
    print(f"sucsess {sucsess} on {day} in {row["latitude"], row['longitude']}")
    if sucsess:
        dates = [(day_date + timedelta(days=i)).strftime("%Y-%m-%d") for i in range(-7, 2)]  # sample every day for a week
            # dates = [(day_date + timedelta(days=i)).strftime("%Y-%m-%d") for i in range(-15, 16)] # get 30 days around that date
    # dates = [(day_date + timedelta(weeks=i)).strftime("%Y-%m-%d") for i in range(-2, 2)]  # sample every week for a month around that date
        for date in dates:
            day_date = datetime.strptime(date, "%Y-%m-%d")
            yesterday = (day_date - timedelta(days=1)).strftime("%Y-%m-%d")
            time_interval=(yesterday, date)
            call_sentinel(client_id,client_secret,row,time_interval,save=True)
            print(f"sucsess {sucsess} on {date} in {row["latitude"], row['longitude']}")
            

Picture not found for location 40.03, 18.0 on 2024-06-14
sucsess False on 2024-06-14 in (40.03, 18.0)
sucsess True on 2024-06-12 in (40.84, 16.13)
Picture not found for location 40.84, 16.13 on 2024-06-05
sucsess True on 2024-06-05 in (40.84, 16.13)
sucsess True on 2024-06-06 in (40.84, 16.13)
sucsess True on 2024-06-07 in (40.84, 16.13)
Picture not found for location 40.84, 16.13 on 2024-06-08
sucsess True on 2024-06-08 in (40.84, 16.13)
Picture not found for location 40.84, 16.13 on 2024-06-09
sucsess True on 2024-06-09 in (40.84, 16.13)
Picture not found for location 40.84, 16.13 on 2024-06-10
sucsess True on 2024-06-10 in (40.84, 16.13)
sucsess True on 2024-06-11 in (40.84, 16.13)
sucsess True on 2024-06-12 in (40.84, 16.13)
Picture not found for location 40.84, 16.13 on 2024-06-13
sucsess True on 2024-06-13 in (40.84, 16.13)
sucsess True on 2024-06-09 in (40.03, 18.0)
Picture not found for location 40.03, 18.0 on 2024-06-02
sucsess True on 2024-06-02 in (40.03, 18.0)
Picture not f

## resize images to be uniform

In [ ]:
# import numpy as np
# 
# def resize_image(input_data, target_size):
#     # input_data should be a numpy array with shape (height, width, 13)
#     height, width, channels = input_data.shape
#     # Create a resized array
#     resized_data = np.zeros((*target_size, channels), dtype=input_data.dtype)
# 
#     # Compute scaling factors
#     scale_height = target_size[0] / height
#     scale_width = target_size[1] / width
# 
#     # Iterate through each pixel and scale the values
#     for h in range(target_size[0]):
#         for w in range(target_size[1]):
#             h_orig = int(h / scale_height)
#             w_orig = int(w / scale_width)
#             resized_data[h, w, :] = input_data[h_orig, w_orig, :]
# 
#     return resized_data
# 
# import os
# import pandas as pd
# from tifffile import imwrite, imread
# 
# # Specify the directory path
# directory_path = 'sentinel_images'  # Replace with your directory path
# # Get a list of all entries (files and subdirectories) in the specified directory
# entries = os.listdir(directory_path)
# # Iterate over each entry
# rows=[]
# for entry in entries:
#     lat,long=entry.split(",")
#     # Create the full path to the entry
#     full_path = os.path.join(directory_path, entry)
# 
#     # Check if the entry is a directory
#     if os.path.isdir(full_path):
#         print(f"Subfolder: {entry}")
# 
#         # List all files and subfolders within the subfolder
#         dates = os.listdir(full_path)
#         for date in dates:
#             rows.append([lat,long,date.split(".")[0]])
#             print(f"  - {date}")
#             file = f"sentinel_images/{entry}/{date}"
#             image = imread(file)
#             resized_image=resize_image(image,(190,190))
#             imwrite(file, resized_image)


# get weather data

## only for dates where we found picture

In [3]:
import os
import pandas as pd
# Specify the directory path
directory_path = 'sentinel_images'  # Replace with your directory path
# Get a list of all entries (files and subdirectories) in the specified directory
entries = os.listdir(directory_path)
# Iterate over each entry
rows=[]
for entry in entries:
    lat,long=entry.split(",")
    # Create the full path to the entry
    full_path = os.path.join(directory_path, entry)

    # Check if the entry is a directory
    if os.path.isdir(full_path):
        print(f"Subfolder: {entry}")

        # List all files and subfolders within the subfolder
        dates = os.listdir(full_path)
        for date in dates:
            rows.append([lat,long,date.split(".")[0]])
            print(f"  - {date}")
    else:
        print(f"File: {entry}")

fire_weather = pd.DataFrame(rows, columns=["latitude","longitude","date"])

Subfolder: 36.68,14.96
  - 2023-07-18.tiff
  - 2023-07-19.tiff
  - 2023-07-23.tiff
  - 2023-07-24.tiff
Subfolder: 36.79,14.68
  - 2022-09-06.tiff
  - 2022-09-07.tiff
  - 2022-09-08.tiff
  - 2022-09-09.tiff
  - 2022-09-11.tiff
  - 2022-09-12.tiff
  - 2022-09-13.tiff
Subfolder: 36.79,14.71
  - 2024-04-23.tiff
  - 2024-04-24.tiff
  - 2024-04-25.tiff
  - 2024-04-26.tiff
  - 2024-04-28.tiff
  - 2024-04-29.tiff
  - 2024-04-30.tiff
  - 2024-05-01.tiff
Subfolder: 36.8,12.04
  - 2023-07-20.tiff
  - 2023-07-21.tiff
  - 2023-07-25.tiff
  - 2023-07-26.tiff
Subfolder: 36.94,14.49
  - 2023-08-17.tiff
  - 2023-08-18.tiff
  - 2023-08-19.tiff
  - 2023-08-20.tiff
  - 2023-08-22.tiff
  - 2023-08-23.tiff
  - 2023-08-24.tiff
  - 2023-08-25.tiff
Subfolder: 36.96,14.53
  - 2023-08-17.tiff
  - 2023-08-18.tiff
  - 2023-08-19.tiff
  - 2023-08-20.tiff
  - 2023-08-22.tiff
  - 2023-08-23.tiff
  - 2023-08-24.tiff
Subfolder: 36.98,14.04
  - 2022-09-04.tiff
  - 2022-09-06.tiff
  - 2022-09-07.tiff
  - 2022-09-08.tiff


In [7]:
import time
from openmeteo_requests.Client import OpenMeteoRequestsError

openmeteo_session=weather.start_openmeteo_session()
def try_get_weather(openmeteo_session,row,time_interval,verbose=False, save=False):
    try:
        weather.get_weather(openmeteo_session,row,time_interval,verbose=False, save=True)
    except OpenMeteoRequestsError as e:
        print(e)
        time.sleep(61)
        try_get_weather(openmeteo_session,row,time_interval,verbose=False, save=True)
for index, row in fire_weather.iterrows():
    print(index)    
    day = row["date"]
    day_date = datetime.strptime(day, "%Y-%m-%d")
    yesterday = (day_date - timedelta(days=1)).strftime("%Y-%m-%d")
    time_interval=(yesterday, day)
    try_get_weather(openmeteo_session,row,time_interval,verbose=False, save=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
def make_labels(fire_data):
    fire_data = fire_data[["latitude", "longitude", "date"]][::-1]
    fire_data.drop_duplicates(inplace=True)
    
    labels=[]
    for _, fire in fire_data:
        labels.append((fire['latitude'], fire['longitude'], fire['date']))